In [33]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import joblib
import sys
import imodels
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
X, y, feature_names = imodels.get_clean_dataset("pima_diabetes")

print("shapes", X.shape, y.shape, "nunique", np.unique(y).size)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.2
)

trans = sklearn.preprocessing.StandardScaler()
X_train = trans.fit_transform(X_train)
X_test = trans.transform(X_test)

m = imodels.AutoInterpretableClassifier()
m.fit(X_train, y_train, cv=3)

print("best params", m.est_.best_params_)
print("best score", m.est_.best_score_)
print("best estimator", m.est_.best_estimator_)
print("best estimator params", m.est_.best_estimator_.get_params())
print('selected from', m.param_grid)

fetching 40715 from openml
shapes (768, 8) (768,) nunique 2


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/sklearn/datasets/_openml.py:971: UserWarning: Version 1 of dataset pima is inactive, meaning that issues have been found in the dataset. Try using a newer version from this URL: https://api.openml.org/data/v1/download/4965316/pima.arff
  warn(


best params {'est': LogisticRegression(C=0.1, l1_ratio=0.5, penalty='elasticnet', solver='saga'), 'est__C': 0.1, 'est__l1_ratio': 0.5}
best score 0.8267026097959889
best estimator Pipeline(steps=[('est',
                 LogisticRegression(C=0.1, l1_ratio=0.5, penalty='elasticnet',
                                    solver='saga'))])
best estimator params {'memory': None, 'steps': [('est', LogisticRegression(C=0.1, l1_ratio=0.5, penalty='elasticnet', solver='saga'))], 'verbose': False, 'est': LogisticRegression(C=0.1, l1_ratio=0.5, penalty='elasticnet', solver='saga'), 'est__C': 0.1, 'est__class_weight': None, 'est__dual': False, 'est__fit_intercept': True, 'est__intercept_scaling': 1, 'est__l1_ratio': 0.5, 'est__max_iter': 100, 'est__multi_class': 'auto', 'est__n_jobs': None, 'est__penalty': 'elasticnet', 'est__random_state': None, 'est__solver': 'saga', 'est__tol': 0.0001, 'est__verbose': 0, 'est__warm_start': False}
selected from [{'est': [DecisionTreeClassifier()], 'est__max_leaf_

In [45]:
d = pd.DataFrame.from_dict({
    'feature_names': feature_names,
    'coef': m.est_.best_estimator_[0].coef_[0]
}).sort_values(by='coef')
vabs = np.max(np.abs(d['coef']))
d.style.background_gradient(cmap=sns.diverging_palette(20, 220, as_cmap=True, center='dark'), vmin=-vabs, vmax=vabs)

,feature_names,coef
2,Diastolic_blood_pressure,-0.134553
4,X_2-Hour_serum_insulin,-0.012109
3,Triceps_skin_fold_thickness,0.000000
7,Age,0.134589
0,Pregnant,0.294840
6,Diabetes_pedigree_function,0.298696
5,Body_mass_index,0.560735
1,plasma_glucose,0.913872
